<a href="https://colab.research.google.com/github/seong576/web1/blob/master/Predict_location_of_retina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Image To Numpy

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from skimage.io import imread
from skimage.transform import pyramid_reduce, resize
import os, glob


path = "/content/drive/MyDrive/retinal/"

img_list = sorted(glob.glob(path+'Train_img/*.jpg'))
mask_list = sorted(glob.glob(path+'Train_Mask/*.png'))
row_size =320
col_size = 320

x_data, y_data = np.empty((2, len(img_list), row_size, col_size , 1), dtype=np.float32)

for i, img_path in enumerate(img_list):
    img = imread(img_path)
    img = resize(img, output_shape=(row_size, col_size , 1), preserve_range=True)
    x_data[i] = img
    
for i, img_path in enumerate(mask_list):
    img = imread(img_path)
    img = resize(img, output_shape=(row_size, col_size , 1), preserve_range=True)
    y_data[i] = img
    
y_data /= 255.

fig, ax = plt.subplots(1, 2)
ax[0].imshow(x_data[12].squeeze(), cmap='gray')
ax[1].imshow(y_data[12].squeeze(), cmap='gray')


x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1)

np.save(path+'x_train.npy', x_train)
np.save(path+'y_train.npy', y_train)
np.save(path+'x_val.npy', x_val)
np.save(path+'y_val.npy', y_val)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

##U-net Model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import print_function
import cv2
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Dropout, Conv2D, Conv3D
from keras.layers import MaxPooling2D, MaxPooling3D, Conv2DTranspose, UpSampling2D
from keras.layers import BatchNormalization, Activation
from keras.layers import Reshape, Dense, multiply, Add, Concatenate
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from keras import models
from keras import callbacks
from keras.layers import Input, Activation, Conv2D, Flatten, Dense, MaxPooling2D, Dropout, Add, LeakyReLU, UpSampling2D
from keras.models import Model, load_model
from keras.callbacks import ReduceLROnPlateau

path = "/content/drive/MyDrive/retinal/"

x_train = np.load(path+'x_train.npy')
y_train = np.load(path+'y_train.npy')
x_val = np.load(path+'x_val.npy')
y_val = np.load(path+'y_val.npy')
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


inputs = Input(shape=(320, 320, 1))


conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2),
                                       padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2),
                                       padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2),
                                       padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2),
                                       padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
model = Model(inputs=[inputs], outputs=[conv10])
model.compile(optimizer=Adam(learning_rate=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

model_checkpoint = ModelCheckpoint('unet.hdf5', monitor='loss', save_best_only=True)

history = model.fit(x_train, y_train, batch_size=2, epochs=20,
                        verbose=1, shuffle=True,
                        validation_data=(x_val, y_val),callbacks=[model_checkpoint])

model.save(path+"Unet")

##Video to Numpy and Prediction

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import cv2
from google.colab.patches import cv2_imshow
path = "/content/drive/MyDrive/retinal/"
mymodel = keras.models.load_model(path+"Unet", compile =False)


cap = cv2.VideoCapture(path+"dhr.mp4")
tensor = []
while cap.isOpened():
    ret, image = cap.read()

    if not ret:
        break
    image = cv2.resize(image, (320, 320))
    gray_img = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    tensor.append(np.asarray(gray_img)[:,:])
    cv2.waitKey(0)

tensor = np.array(tensor)
cv2.destroyAllWindows()
cap.release()

x_test = tensor
#y_test = np.load(path+"y_test.npy")
preds = model.predict(x_test)


img_rows =320
img_cols =320
def set_view():
    total=x_test.shape[0]
    print (x_test.shape)
    dt_img = np.ndarray((total, img_rows, img_cols), dtype=np.uint8)
    rgb_img = np.ndarray((total, img_rows, img_cols, 3), dtype=np.uint8)
    for m in range(total):
        gray_img = x_test[m]
        gray_img = cv2.resize(gray_img, (img_cols, img_rows))

        t_img = preds[m,:,:,0]
        t_img = cv2.resize(t_img, (img_cols, img_rows))

        kernel = np.ones((3,3), np.uint8)
        dilated_img = cv2.dilate(128*t_img, kernel, iterations=1)

        boundary_img = dilated_img * (1-t_img)

        rgb_img[m][:,:,0] = gray_img
        rgb_img[m][:,:,1] = np.clip(gray_img + boundary_img, 0, 255)
        rgb_img[m][:,:,2] = gray_img
    return rgb_img

view_imgs ,_ = set_view()

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(path+"동영상r.mp4", fourcc, 30, (img_rows,img_cols))
print(view_imgs.shape)
for i in range(len(view_imgs)):
      out.write(np.uint8(view_imgs[i]))
out.release()
cv2.destroyAllWindows()
